In [ ]:
import pandas as pd
import pandahouse as ph
import datetime as dt
import seaborn as sns
import datetime as dt
from scipy import stats as st
import telegram
import matplotlib.pyplot as plt
import io
from airflow.decorators import dag, task

In [ ]:
default_args = {
    'owner': 'name',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': dt.timedelta(minutes=5),
    'start_date': dt.datetime(2022, 7, 30),
}

In [ ]:
my_token = '<token>'
bot = telegram.Bot(token=my_token)

In [ ]:
schedule_interval = '15 * * * *'
connection = {'host': 'host',
                      'database':'db',
                      'user':'name', 
                      'password':'password'
                     }

In [ ]:
q = """
SELECT COUNT(DISTINCT user_id) as DAU,
countIf(action = 'like') as likes,
countIf(action = 'view') as views,
likes/views as CTR,
source,
toStartOfFifteenMinutes(time) as fifteen_minutes,
DATE(time) as day
FROM simulator_20220720.feed_actions
WHERE DATE(time) BETWEEN DATE(today() - 365) and DATE(today())
GROUP BY toStartOfFifteenMinutes(time), day, source
"""

In [ ]:
q1 = """
SELECT COUNT(user_id) as messages,
COUNT(DISTINCT user_id) as sendlers,
COUNT(DIStiNCT reciever_id) as recievers,
source,
DATE(time) as day,
toStartOfFifteenMinutes(time) as fifteen_minutes
FROM simulator_20220720.message_actions
WHERE DATE(time) BETWEEN DATE(today() - 365) and DATE(today())
GROUP BY day, fifteen_minutes, source
"""

In [ ]:
today = dt.datetime.now().strftime(format='%Y-%m-%d')
alpha = 0.05

In [ ]:
@dag(default_args=default_args, schedule_interval=schedule_interval, catchup=False)
def lmk_report_8():
    
    @task
    def actions():
        actions = ph.read_clickhouse(q, connection=connection)
        actions['minutes'] = actions['fifteen_minutes'].apply(lambda x: x.strftime(format='%H:%M'))
        return actions
    
    @task
    def messages():
        messages = ph.read_clickhouse(q1, connection=connection)
        messages['minutes'] = messages['fifteen_minutes'].apply(lambda x: x.strftime(format='%H:%M'))
        return messages
    
    @task
    def actions_alert(actions):
        last_fm = actions.query('day == @today').sort_values(by='minutes').tail(1)['minutes'].unique()[0]
        metrics = []
        for i in ['DAU', 'likes', 'views', 'CTR']:
            for j in ['organic', 'ads']:
                interested_value = actions.query('day == @today and source == @j and minutes == @last_fm')[i].unique()[0]
                query = actions.query('day < @today and source == @j and minutes == @last_fm')[i]
                results = st.ttest_1samp(query, interested_value)
                if results.pvalue < alpha:
                    metrics.append([i, j, interested_value, '{:.2f}'.format(query.mean())])
        return metrics
        
    @task
    def messages_alert(messages):
        last_fm = messages.query('day == @today').sort_values(by='minutes').tail(1)['minutes'].unique()[0]
        metrics = []
        for i in ['messages', 'sendlers', 'recievers']:
            for j in ['organic', 'ads']:
                interested_value = messages.query('day == @today and source == @j and minutes == @last_fm')[i].unique()[0]
                query =  messages.query('day < @today and source == @j and minutes == @last_fm')[i]
                results = st.ttest_1samp(query, interested_value)
                if results.pvalue < alpha:
                    metrics.append([i, j, interested_value, '{:.2f}'.format(query.mean())])
        return metrics
    
    @task
    def metrics_msg(actions_metrics, messages_metrics, chat_id):
        msg = []
        actions_metrics = actions_metrics
        messages_metrics = messages_metrics
        if len(actions_metrics) == 0:
            if len(messages_metrics) == 0:
                return 'all right'
            else:
                for i in messages_metrics:
                    msg.append('Метрика: {}, источник трафика: {}\nзначение: {}, отклонение от среднегодового значения: {:.2%}'.format(i[0], i[1], i[2], (float(i[2]) - float(i[3]))/float(i[3])))
        else:
            for i in actions_metrics:
                    msg.append('Метрика: {}, источник трафика: {}\nзначение: {}, отклонение от среднегодового значения: {:.2%}'.format(i[0], i[1], i[2], (float(i[2]) - float(i[3]))/float(i[3])))
        if len(msg) != 0:
            msg = '\n'.join(msg)
            bot.sendMessage(chat_id=chat_id, text=msg)
            
    @task
    def plot_msg(df, df_metrics, chat_id):
        if len(df_metrics) != 0:
            last_fm = df.query('day == @today').sort_values(by='minutes').tail(1)['minutes'].unique()[0]
            metrics = list(set([i[0] for i in df_metrics]))
            fig, ax = plt.subplots(len(metrics))
            for i in range(len(metrics)):
                fig.autofmt_xdate()
                sns.lineplot(ax=ax[i], x='day', y=df[metrics[i]], legend='full', data=df.query('minutes == @last_fm'))
            plot_object = io.BytesIO()
            plt.savefig(plot_object)
            plot_object.seek(0)
            plot_object.name = 'plot.png'
            plt.close()
            bot.sendPhoto(chat_id=chat_id, photo=plot_object)
                
    actions = actions()
    actions_metrics = actions_alert(actions)
    messages = messages()
    messages_metrics = messages_alert(messages)        
    metrics_msg(actions_metrics, messages_metrics, chat_id='<id>') 
    plot_msg(actions, actions_metrics, chat_id='<id>')
    plot_msg(messages, messages_metrics, chat_id='<id>')

In [ ]:
lmk_report_8 = lmk_report_8()